# ValuePlayer

Der ValuePlayer ist eine abstrakte Subklasse von Player und benötigt zusätzlich noch eine Value-Funktion die Spielbrettposition bewertet und dann den Zug nimmt der die Value-Funktion maximiert.

Dadurch lassen sich viele einfache Strategien repräsentieren

In [1]:
from Player import Player

class ValuePlayer(Player):
    
    def __init__(self, player, valuefunction):
        Player.__init__(self, player)
        self.value = valuefunction
        
    def get_action(self, actions, game):
        # Spielstatus speichern
        old_state = game.get_state()
        # Variablen initialisieren
        best_value = float("-inf")
        best_action = None
        # Alle Züge durchsuchen
        for a in actions:
            # Zug ausführen
            game.execute_moves(a, self.player)
            # Spielstatus bewerten
            value = self.value(game, self.player)
            # Besten merken
            if value > best_value:
                best_value = value
                best_action = a
            # Spiel zurücksetzen
            game.reset_to_state(old_state)
        return best_action
    
    def get_name(self):
        return "ValuePlayer [" + self.value.__name__ + "]"

## ValueFunction
Eine Funktion die eine Zahl zurückgibt, die dann vom ValuePlayer maximiert wird.

### WayToGo Funktion
Die WayToGo Funktion zählt die Schritte die nötig sind um alle Steine "ins Ziel" zu bringen. <br>
Dazu berechnet er GegnerSchritte - EigeneSchritte, sodass man große Werte positiv für den Spieler sind.

Damit man eine Zahl zwischen 0 und 1 erhält teilt man durch den worst-case (alle 15 Steine auf der bar) 15*25 und addiert 0.5

#### Optimierung: 
Egal welchen Zug man ausführt, die eigenen Schritte werden immer gleich sein. <br>
Nur die Gegnerschritte können durch schlagen erhöht werden <br>
-> Nur Gegnerschritte zählen <br>
-> Ergebnis muss dann nur noch durch 15*25 (375) geteilt werden ohne 0.5 zu addieren

In [2]:
def way_to_go(game, player):
    # Seite ermitteln
    black = player == game.players[0]
    # Steine besorgen
    enemy_checkers = game.white_checkers if black else game.black_checkers
    # Schritte berechnen
    steps = 0
    # Steine auf dem Spielfeld
    for tower in enemy_checkers:
        if black:
            steps += (24 - tower) * game.points[tower]
        else:
            steps += tower * game.points[tower] * -1
    # Steine auf der Bar
    bar_chk = game.white_taken if black else game.black_taken
    steps += 25 * bar_chk 
    return steps / 375

#### Testfunktion um zwei Player gegeneinander antreten zu lassen

In [3]:
import time
from FasterBackgammon import Game

def test(players, games=200, debug=False):
    wins = {Game.PLAYERS[0] : 0, Game.PLAYERS[1] : 0}
    # Zeit messen und Spielen, diesmal ohne loggen
    start = time.time()
    for i in range(games):
        game = Game()
        winner = game.play(players, debug=debug)
        wins[winner] += 1
        win_num = 0 if winner == game.players[0] else 1
        print("Spiel", i, "von", games ,"geht an", players[win_num].get_name(), "(" , winner , ")")
    end = time.time()

    # Hübsch ausgeben
    print()
    print(wins)
    print(games, "Spiele in ", end - start, "Sekunden")

#### WayToGo vs Random

In [4]:
from Player import RandomPlayer

players = [ValuePlayer(Game.PLAYERS[0], way_to_go), RandomPlayer(Game.PLAYERS[1])]
test(players)

Spiel 0 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 1 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 2 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 3 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 4 von 200 geht an RandomPlayer ( white )
Spiel 5 von 200 geht an RandomPlayer ( white )
Spiel 6 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 7 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 8 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 9 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 10 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 11 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 12 von 200 geht an RandomPlayer ( white )
Spiel 13 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 14 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 15 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 16 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 17 von 200 geht an RandomPlayer ( 

Spiel 145 von 200 geht an RandomPlayer ( white )
Spiel 146 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 147 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 148 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 149 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 150 von 200 geht an RandomPlayer ( white )
Spiel 151 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 152 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 153 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 154 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 155 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 156 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 157 von 200 geht an RandomPlayer ( white )
Spiel 158 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 159 von 200 geht an RandomPlayer ( white )
Spiel 160 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 161 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 162 von 200 geht a

#### WayToGo vs WayToGo

In [6]:
players = [ValuePlayer(Game.PLAYERS[0], way_to_go), ValuePlayer(Game.PLAYERS[1], way_to_go)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 1 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 2 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 3 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 4 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 5 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 6 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 7 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 8 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 9 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 10 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 11 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 12 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 13 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 14 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 15 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 16 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 1

Spiel 139 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 140 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 141 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 142 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 143 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 144 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 145 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 146 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 147 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 148 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 149 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 150 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 151 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 152 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 153 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 154 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 155 von 200 geht an ValuePlayer [w

WayToGo scheint bei jeder Gelegenheit Gegner rauswerfen zu wollen. Dadurch ziehen sich die Spiele in die Länge

### Singleton Funktion
Zählt die Anzahl der einzelnen Steine auf dem Spielfeld. Da einzelne Steine rausgeworfen werden können, versucht man den Wert 15 - einzelne Steine zu maximieren 

In [16]:
def singleton(game, player):
    chk = game.black_checkers if player == game.players[0] else game.white_checkers
    singles = 0
    for i in chk:
        if abs(game.points[i]) == 1:
            singles += 1
            
    # Zahl in [0,1]
    return (15 - singles) / 15

#### Singleton vs Random

In [42]:
players = [ValuePlayer(Game.PLAYERS[0], singleton), RandomPlayer(Game.PLAYERS[1])]
test(players)

Spiel 0 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 1 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 2 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 3 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 4 von 200 geht an RandomPlayer ( white )
Spiel 5 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 6 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 7 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 8 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 9 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 10 von 200 geht an RandomPlayer ( white )
Spiel 11 von 200 geht an RandomPlayer ( white )
Spiel 12 von 200 geht an RandomPlayer ( white )
Spiel 13 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 14 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 15 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 16 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 17 von 200 geht an ValuePlayer [singleton] ( 

Spiel 163 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 164 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 165 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 166 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 167 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 168 von 200 geht an RandomPlayer ( white )
Spiel 169 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 170 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 171 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 172 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 173 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 174 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 175 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 176 von 200 geht an RandomPlayer ( white )
Spiel 177 von 200 geht an RandomPlayer ( white )
Spiel 178 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 179 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 180 von

#### Singleton vs Singleton

In [43]:
players = [ValuePlayer(Game.PLAYERS[0], singleton), ValuePlayer(Game.PLAYERS[1], singleton)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 1 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 2 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 3 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 4 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 5 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 6 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 7 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 8 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 9 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 10 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 11 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 12 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 13 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 14 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 15 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 16 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 1

Spiel 149 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 150 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 151 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 152 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 153 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 154 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 155 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 156 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 157 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 158 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 159 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 160 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 161 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 162 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 163 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 164 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 165 von 200 geht an ValuePlayer [s

Singleton spielt vorsichtig aber effektiv.

Mal sehen wie das defensive Singleton mit dem aggressiven WayToGo klar kommt

#### Singleton vs WayToGo

In [44]:
players = [ValuePlayer(Game.PLAYERS[0], singleton), ValuePlayer(Game.PLAYERS[1], way_to_go)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 1 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 2 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 3 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 4 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 5 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 6 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 7 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 8 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 9 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 10 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 11 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 12 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 13 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 14 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 15 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 16 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 1

Spiel 139 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 140 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 141 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 142 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 143 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 144 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 145 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 146 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 147 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 148 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 149 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 150 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 151 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 152 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 153 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 154 von 200 geht an ValuePlayer [singleton] ( black )
Spiel 155 von 200 geht an ValuePlayer [w

Das aggessive WayToGo findet gegen den vorsichtigen singleton keine Angriffsfläche

In [45]:
players = [ValuePlayer(Game.PLAYERS[0], way_to_go), ValuePlayer(Game.PLAYERS[1], singleton)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 1 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 2 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 3 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 4 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 5 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 6 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 7 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 8 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 9 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 10 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 11 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 12 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 13 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 14 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 15 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 16 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 1

Spiel 139 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 140 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 141 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 142 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 143 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 144 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 145 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 146 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 147 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 148 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 149 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 150 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 151 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 152 von 200 geht an ValuePlayer [way_to_go] ( black )
Spiel 153 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 154 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 155 von 200 geht an ValuePlayer [s

#### Bug?
Bei Singleton vs WayToGo ist es ausgeglichen und bei WayToGo vs Singleton gewinnt singleton deutlich...

Dabei spielen jeweils beide ausgeglichen gegen sich selber

### WayToGo + Singleton kombiniert

Da beide Zahlen zwischen 0 und 1 produzieren kann man sie einfach addieren um beide Strategiern auszuführen

In [24]:
def single_to_go(game, player):
    return singleton(game, player) + way_to_go(game, player)
    

players = [ValuePlayer(Game.PLAYERS[0], single_to_go), ValuePlayer(Game.PLAYERS[1], singleton)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 1 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 2 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 3 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 4 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 5 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 6 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 7 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 8 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 9 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 10 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 11 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 12 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 13 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 14 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 15 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 16 von 200 geht an ValuePlayer [single_to_go] ( 

Spiel 138 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 139 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 140 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 141 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 142 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 143 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 144 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 145 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 146 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 147 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 148 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 149 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 150 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 151 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 152 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 153 von 200 geht an ValuePlayer [single_to_go] ( black )
S

In [47]:
players = [ValuePlayer(Game.PLAYERS[0], single_to_go), ValuePlayer(Game.PLAYERS[1], way_to_go)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 1 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 2 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 3 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 4 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 5 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 6 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 7 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 8 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 9 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 10 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 11 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 12 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 13 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 14 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 15 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 16 von 200 geht an ValuePla

Spiel 138 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 139 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 140 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 141 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 142 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 143 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 144 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 145 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 146 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 147 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 148 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 149 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 150 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 151 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 152 von 200 geht an ValuePlayer [single_to_go] ( black )
Spiel 153 von 200 geht an ValuePlayer [single_to_go] ( black 

## Ergebnis

__SingleToGo__ > __Singleton__ > __WayToGo__

<ul>
    <li>WayToGo hat eine verhältnismäßig komplizierte Funktion, spielt am schlechtesten und am langsamsten, da es nur den Gegner rauswerfen will ohne slebst wirklich Fortschritt zu machen </li>

<li>Singleton hat eine einfache Funktion, es spielt gut und schnell </li>
<li>single_to_go spielt besser als singleton, da es auch aggressive Züge ausführt die den Gegner weit zurückwerfen, spielt aber auch deutlich schnellere Spiele dank dem Singleton Anteil </li>
<ul><li>Im Endeffekt geringfügig besser als singleton</li></ul>
</ul>

### Blocker

Ein Spieler der versucht, möglichst viele Felder mit je zwei Steinen (am besten nebeneinander) zu belegen, so dass der Gegner dort nicht hin ziehen kann. <br>
Da es 15 Steine gibt und man mindestens 2 Steine braucht um ein Feld zu blockieren, kann man maximal 7 2er Türme haben.

Value = Anzahl der 2er Türme / 7

In [12]:
def blocker(game, player):
    chk = game.black_checkers if player == game.players[0] else game.white_checkers
    blocked = len([x for x in chk if abs(game.points[x]) >= 2])
    return blocked / 7

#### Blocker vs Random

In [13]:
from Player import RandomPlayer

players = [ValuePlayer(Game.PLAYERS[0], blocker), RandomPlayer(Game.PLAYERS[1])]
test(players)

Spiel 0 von 200 geht an RandomPlayer ( white )
Spiel 1 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 2 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 3 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 4 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 5 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 6 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 7 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 8 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 9 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 10 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 11 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 12 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 13 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 14 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 15 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 16 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 17 von 200 geht an ValuePlayer [blocker] ( b

Spiel 148 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 149 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 150 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 151 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 152 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 153 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 154 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 155 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 156 von 200 geht an RandomPlayer ( white )
Spiel 157 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 158 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 159 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 160 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 161 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 162 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 163 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 164 von 200 geht an RandomPlayer ( white )
Spiel 165 von 200 geht an ValueP

#### Blocker vs Blocker

In [14]:
players = [ValuePlayer(Game.PLAYERS[0], blocker), ValuePlayer(Game.PLAYERS[1], blocker)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 1 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 2 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 3 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 4 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 5 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 6 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 7 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 8 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 9 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 10 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 11 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 12 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 13 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 14 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 15 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 16 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 17 von 200 geht an ValuePlayer [blo

Spiel 145 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 146 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 147 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 148 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 149 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 150 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 151 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 152 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 153 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 154 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 155 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 156 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 157 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 158 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 159 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 160 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 161 von 200 geht an ValuePlayer [blocker] ( white )
Spiel 162 von 

#### Blocker vs WayToGo

In [15]:
players = [ValuePlayer(Game.PLAYERS[0], blocker), ValuePlayer(Game.PLAYERS[1], way_to_go)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 1 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 2 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 3 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 4 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 5 von 200 geht an ValuePlayer [way_to_go] ( white )
Spiel 6 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 7 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 8 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 9 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 10 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 11 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 12 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 13 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 14 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 15 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 16 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 17 von 200 geht an ValuePlaye

Spiel 145 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 146 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 147 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 148 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 149 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 150 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 151 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 152 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 153 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 154 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 155 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 156 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 157 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 158 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 159 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 160 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 161 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 162 von 

#### Blocker vs Singleton

In [19]:
players = [ValuePlayer(Game.PLAYERS[0], blocker), ValuePlayer(Game.PLAYERS[1], singleton)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 1 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 2 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 3 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 4 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 5 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 6 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 7 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 8 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 9 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 10 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 11 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 12 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 13 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 14 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 15 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 16 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 17 von 200 geht an Valu

Spiel 146 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 147 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 148 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 149 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 150 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 151 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 152 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 153 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 154 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 155 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 156 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 157 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 158 von 200 geht an ValuePlayer [blocker] ( black )
Spiel 159 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 160 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 161 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 162 von 200 geht an ValuePlayer [blocker] ( black )
Sp

Blocker gegen Singleton ist sehr ausgeglichen was daran liegt, dass die Ideen zu ähnlichen Ergebnissen führen:

Wenn man versucht möglichst viele 2er Türme zu bauen hat man keine einzelnen Steine mehr, aber auch wenn man versucht keine einzelnen Steine mehr zu haben bilden sich automatisch 2er Türme

### Weiteres (gewichtetes) kombinieren

Oben wurde Singleton mit way_to_go kombiniert um eine neue Funktion zu erschaffen. Das kann man natürlich mit anderen Funktionen auch machen und noch entsprechend die Funktionen einzelnt Gewichten.

In [31]:
def blocker_to_go(game, player):
    return blocker(game, player) + way_to_go(game, player) * 1.5

players = [ValuePlayer(Game.PLAYERS[0], blocker_to_go), ValuePlayer(Game.PLAYERS[1], singleton)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 1 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 2 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 3 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 4 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 5 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 6 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 7 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 8 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 9 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 10 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 11 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 12 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 13 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 14 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 15 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 16 von 200 geht an V

Spiel 141 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 142 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 143 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 144 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 145 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 146 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 147 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 148 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 149 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 150 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 151 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 152 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 153 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 154 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 155 von 200 geht an ValuePlayer [singleton] ( white )
Spiel 156 von 200 geht an ValuePlayer [singleton] ( white )


In [32]:
players = [ValuePlayer(Game.PLAYERS[0], blocker_to_go), ValuePlayer(Game.PLAYERS[1], single_to_go)]
test(players)

Spiel 0 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 1 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 2 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 3 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 4 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 5 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 6 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 7 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 8 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 9 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 10 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 11 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 12 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 13 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 14 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 15 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 16 v

Spiel 131 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 132 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 133 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 134 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 135 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 136 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 137 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 138 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 139 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 140 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 141 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 142 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 143 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 144 von 200 geht an ValuePlayer [blocker_to_go] ( black )
Spiel 145 von 200 geht an ValuePlayer [single_to_go] ( white )
Spiel 146 von 200 geht an ValuePlayer [singl